In [1]:
import pickle
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import folium

In [2]:
# Load Data with Pickle
infile = open('data/inicialesspain', 'rb')
infile2 = open('data/finalesspain', 'rb')

planned_data_flights = pickle.load(infile)
real_data_flights = pickle.load(infile2)



In [ ]:
# Global variables
df = pd.read_pickle('spanish_airports.pkl')
spanish_ICAO_codes = set(df['ICAO'].tolist())

# ---------- PARSER ----------
def parse_coord_line(line_route):
    splited_point = line_route.split(":")
    
    time_str = splited_point[0]
    alt_str = splited_point[3]
    coord_str = splited_point[6]
    airport_str = splited_point[1]

    dt = datetime.strptime(time_str, "%Y%m%d%H%M%S")
    alt = int(alt_str) * 100 * 0.0003048  # Feet to KM

    def dms_to_decimal(dms, direction):
        degrees, minutes, seconds = 0, 0, 0
        if direction in ["N", "S"]:
            degrees = int(dms[:2])
            minutes = int(dms[2:4])
            seconds = int(dms[4:6])
        elif direction in ["E", "W"]:
            degrees = int(dms[:3])
            minutes = int(dms[3:5])
            seconds = int(dms[5:7])
        decimal = degrees + minutes / 60 + seconds / 3600
        return -decimal if direction in ["S", "W"] else decimal

    lat = dms_to_decimal(coord_str[:6], coord_str[6])
    lon = dms_to_decimal(coord_str[7:14], coord_str[14])

    return (lat, lon, alt, dt, airport_str)

# ---------- MAP FLIGHTS ----------
def map_flight(real_flight, planned_flight, final_flights_dict, stats):
    stats["total"] += 1
    
    dep_airport, arr_airport, fid = planned_flight[:3]
    if dep_airport == arr_airport:
        stats["circular"] += 1
        return None
    
    if "ZZZZ" in [dep_airport, arr_airport]:
        stats["zzzz"] += 1
        return None

    real_parsed = [parse_coord_line(line) for line in real_flight[3].split()]
    plan_parsed = [parse_coord_line(line) for line in planned_flight[3].split()]
    
    if real_parsed == plan_parsed:
        stats["perfect"] += 1

    # Check canceled flights
    if len(real_parsed) < 3:
        stats["cancelled"] += 1
        return None
    
    # Check diverted flights
    if real_parsed[-1][4] != arr_airport:
        stats["diverted"] += 1
        return None
    
    # Check bad plan flights
    if not (plan_parsed[0][4] == dep_airport) or not(plan_parsed[-1][4] == arr_airport):
        stats["bad_plan"] += 1
        return None

    # Remove airport info
    real_parsed = [p[:4] for p in real_parsed]
    plan_parsed = [p[:4] for p in plan_parsed]

    if ((plan_parsed[0][3] - real_parsed[0][3]).total_seconds() / 60 > 20) and ((plan_parsed[-1][3] - real_parsed[-1][3]).total_seconds() / 60 > 60):
        stats["much_before_planned"] += 1
        return None

    # Check duplicated timestamps
    def has_duplicates(traj):
        seen = {}
        for lat, lon, alt, dt in traj:
            if dt not in seen:
                seen[dt] = set()
            seen[dt].add((lat, lon, alt))
            if len(seen[dt]) > 1:
                return True
        return False

    # Check duplicated real timestamps
    if has_duplicates(real_parsed):
        stats["dup_dt_real"] += 1
        return None
    
    # Check duplicated plan timestamps
    if has_duplicates(plan_parsed):
        stats["dup_dt_plan"] += 1
        return None

    # Hash Key
    # flight_key = hash(tuple(plan_parsed))
    flight_key = f"{fid}_{hash(tuple(plan_parsed))}"
    
    # Filter spanish airports
    dep_airport = dep_airport if dep_airport in spanish_ICAO_codes else None
    arr_airport = arr_airport if arr_airport in spanish_ICAO_codes else None
    
    date = real_parsed[-1][3].strftime('%d/%m/%Y')
    
    # Remove flights which arrived before or after the days
    if date == '24/03/2018' or date == '28/10/2018':
        stats["bad_date"] += 1
        return None

    # Check Duplicates
    if flight_key in final_flights_dict:
        stats["duplicated"] += 1
        existing = final_flights_dict[flight_key]
        if len(real_parsed) > len(existing["real_route"]):
            final_flights_dict[flight_key]["real_route"] = real_parsed
            stats["updated"] += 1
    else:
        final_flights_dict[flight_key] = {
            "flight_id": fid,
            "date": date,
            "dep_airport": dep_airport,
            "arr_airport": arr_airport,
            "plan_route": plan_parsed,
            "real_route": real_parsed
        }

    return final_flights_dict[flight_key]

# ---------- EXECUTION IN BLOCKS ----------
def process_flights(real_data_flights, planned_data_flights, save_every=10):
    final_flights_dict = {}
    stats = {
        "total": 0,"circular": 0,"zzzz": 0,"perfect": 0, "cancelled": 0, "diverted": 0,"bad_plan": 0,
        "dup_dt_real": 0, "dup_dt_plan": 0,"bad_date": 0, "duplicated": 0, "updated": 0, "much_before_planned": 0
    }

    for i in range(len(real_data_flights)):
        if i % 1 == 0:
            print(f"Processing day {i}...")

        for real_flight, planned_flight in zip(real_data_flights[i], planned_data_flights[i]):
            try:
                map_flight(real_flight, planned_flight, final_flights_dict, stats)
            except Exception as e:
                print(f"Error in flight: {e}")
                continue

    return final_flights_dict, stats


In [4]:
final_flights_dict, stats = process_flights(real_data_flights, planned_data_flights)
print("\n--------------------------------------------")
print("Final flights:", len(final_flights_dict))
for k, v in stats.items():
    print(f"{k}: {v}")


Processing day 0...
Processing day 1...
Processing day 2...
Processing day 3...
Processing day 4...
Processing day 5...
Processing day 6...
Processing day 7...
Processing day 8...
Processing day 9...
Processing day 10...
Processing day 11...
Processing day 12...
Processing day 13...
Processing day 14...
Processing day 15...
Processing day 16...
Processing day 17...
Processing day 18...
Processing day 19...
Processing day 20...
Processing day 21...
Processing day 22...
Processing day 23...
Processing day 24...
Processing day 25...
Processing day 26...
Processing day 27...
Processing day 28...
Processing day 29...
Processing day 30...
Processing day 31...
Processing day 32...
Processing day 33...
Processing day 34...
Processing day 35...
Processing day 36...
Processing day 37...
Processing day 38...
Processing day 39...
Processing day 40...
Processing day 41...
Processing day 42...
Processing day 43...
Processing day 44...
Processing day 45...
Processing day 46...
Processing day 47...
Pr

In [5]:
print("Final flights:", len(final_flights_dict))
for k, v in stats.items():
    print(f"{k}: {v}")


Final flights: 1098335
total: 1134813
circular: 4219
zzzz: 202
perfect: 0
cancelled: 25
diverted: 1518
bad_plan: 0
dup_dt_real: 202
dup_dt_plan: 0
bad_date: 193
duplicated: 29869
updated: 22435
much_before_planned: 250


--------------------------------------------
Final flights: 1098440
total: 1134813
circular: 4219
cancelled: 25
diverted: 1518
duplicated: 30013
updated: 22486
zzzz: 202
bad_plan: 0
dup_dt_real: 203
dup_dt_plan: 0
perfect: 0
bad_date: 193

In [6]:
removed = stats["circular"] + stats["cancelled"] + stats["diverted"] + stats["zzzz"] + stats["bad_plan"] + stats["dup_dt_real"] + stats["bad_date"] + stats["duplicated"]
removed2 = stats["total"] - len(final_flights_dict)
print(f'Total flights removed: {removed}, {removed2}')

Total flights removed: 36228, 36478


In [7]:
# Convertir final_flights_dict a una lista de diccionarios
flights_data = list(final_flights_dict.values())

# Crear el DataFrame
df = pd.DataFrame(flights_data)

print(df.head())

           flight_id        date dep_airport arr_airport  \
0  AFL2605AA73604012  25/03/2018        LEMD        None   
1  AFL2529AA73604073  25/03/2018        LEMG        None   
2  AFL2513AA73605040  25/03/2018        LEBL        None   
3  AFL2549AA73600581  25/03/2018        GCTS        None   
4  NAX1715AA73579261  25/03/2018        GCLP        None   

                                          plan_route  \
0  [(40.47222222222222, -3.560833333333333, 0.0, ...   
1  [(36.675, -4.4991666666666665, 0.0, 2018-03-24...   
2  [(41.29694444444444, 2.0783333333333336, 0.0, ...   
3  [(28.044444444444448, -16.572499999999998, 0.0...   
4  [(27.931944444444447, -15.386666666666667, 0.0...   

                                          real_route  
0  [(40.47222222222222, -3.560833333333333, 0.0, ...  
1  [(36.675, -4.4991666666666665, 0.0, 2018-03-24...  
2  [(41.29694444444444, 2.0783333333333336, 0.0, ...  
3  [(28.044444444444448, -16.572499999999998, 0.0...  
4  [(27.931944444444447, -1

In [8]:
# Example: first flight
first_flight = df.iloc[1]
flight_id = first_flight["flight_id"]
date = first_flight["date"]
dep_airport = first_flight["dep_airport"]
arr_airport = first_flight["arr_airport"]

plan_route = first_flight["plan_route"]
real_route = first_flight["real_route"]



print(f"Flight Id: {flight_id}\n")
print(f"Date: {date}\n")
print(f"Departure: {dep_airport}\n")
print(f"Arrival: {arr_airport}\n")

print(f"Plan Route:") 
for point in plan_route:
    print(point)
    
print(f"\nReal Route:") 
for point in real_route:
    print(point)

Flight Id: AFL2529AA73604073

Date: 25/03/2018

Departure: LEMG

Arrival: None

Plan Route:
(36.675, -4.4991666666666665, 0.0, datetime.datetime(2018, 3, 24, 22, 50))
(36.68083333333333, -4.5072222222222225, 0.15239999999999998, datetime.datetime(2018, 3, 24, 22, 50, 11))
(36.715, -4.5552777777777775, 1.0668, datetime.datetime(2018, 3, 24, 22, 51, 29))
(36.72638888888889, -4.5713888888888885, 1.31064, datetime.datetime(2018, 3, 24, 22, 51, 45))
(36.74444444444445, -4.574722222222222, 1.5239999999999998, datetime.datetime(2018, 3, 24, 22, 52))
(36.83416666666667, -4.591111111111111, 2.7432, datetime.datetime(2018, 3, 24, 22, 53, 23))
(36.905833333333334, -4.604444444444444, 3.3528, datetime.datetime(2018, 3, 24, 22, 54, 6))
(36.93277777777777, -4.609444444444444, 3.6271199999999997, datetime.datetime(2018, 3, 24, 22, 54, 26))
(36.963055555555556, -4.585, 3.9623999999999997, datetime.datetime(2018, 3, 24, 22, 54, 52))
(37.18277777777777, -4.4077777777777785, 5.7912, datetime.datetime(201

In [9]:
df_dates = df
df_dates['date'] = pd.to_datetime(df_dates['date'])
count_dates = df_dates['date'].value_counts().reset_index()
count_dates.columns = ['date', 'count']
count_dates = count_dates.sort_values(by='date')
for row in count_dates.iterrows():
    print(row[1]['date'], row[1]['count'])

C:\Users\Raúl\AppData\Local\Temp\ipykernel_16696\2280983662.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_dates['date'] = pd.to_datetime(df_dates['date'])


2018-03-25 00:00:00 4450
2018-03-26 00:00:00 4242
2018-03-27 00:00:00 4128
2018-03-28 00:00:00 4308
2018-03-29 00:00:00 4479
2018-03-30 00:00:00 4271
2018-03-31 00:00:00 4534
2018-04-01 00:00:00 4693
2018-04-02 00:00:00 4591
2018-04-03 00:00:00 1824
2018-04-04 00:00:00 4329
2018-04-05 00:00:00 4387
2018-04-06 00:00:00 4559
2018-04-07 00:00:00 4509
2018-04-08 00:00:00 4635
2018-04-09 00:00:00 4414
2018-04-10 00:00:00 4286
2018-04-11 00:00:00 4222
2018-04-12 00:00:00 4382
2018-04-13 00:00:00 4547
2018-04-14 00:00:00 4574
2018-04-15 00:00:00 4603
2018-04-16 00:00:00 4391
2018-04-17 00:00:00 4253
2018-04-18 00:00:00 4273
2018-04-19 00:00:00 4337
2018-04-20 00:00:00 4555
2018-04-21 00:00:00 4543
2018-04-22 00:00:00 4575
2018-04-23 00:00:00 4433
2018-04-24 00:00:00 4360
2018-04-25 00:00:00 4192
2018-04-26 00:00:00 4423
2018-04-27 00:00:00 4761
2018-04-28 00:00:00 4690
2018-04-29 00:00:00 4514
2018-04-30 00:00:00 4477
2018-05-01 00:00:00 4572
2018-05-02 00:00:00 4534
2018-05-03 00:00:00 4432


In [10]:
import os
import pandas as pd

def save_dataframe_by_date_blocks(df, output_folder="df_by_date_blocks", block_size=10):
    """
    Saves the DataFrame into multiple pickle files, each containing rows
    corresponding to 10 unique dates from the 'date' column.
    """
    os.makedirs(output_folder, exist_ok=True)

    # Get sorted list of unique dates
    unique_dates = sorted(df["date"].unique())
    print(f"Total unique dates: {len(unique_dates)}")

    for i in range(0, len(unique_dates), block_size):
        date_block = unique_dates[i:i + block_size]
        df_block = df[df["date"].isin(date_block)]

        filename = f"block_{i // block_size + 1:02d}.pkl"
        filepath = os.path.join(output_folder, filename)

        df_block.to_pickle(filepath)
        print(f"Saved: {filepath} → {df_block.shape[0]} rows")

save_dataframe_by_date_blocks(df, output_folder="data_by_dates", block_size=10)


Total unique dates: 217
Saved: data_by_dates\block_01.pkl → 41520 rows
Saved: data_by_dates\block_02.pkl → 44270 rows
Saved: data_by_dates\block_03.pkl → 44537 rows
Saved: data_by_dates\block_04.pkl → 44955 rows
Saved: data_by_dates\block_05.pkl → 47677 rows
Saved: data_by_dates\block_06.pkl → 47354 rows
Saved: data_by_dates\block_07.pkl → 49764 rows
Saved: data_by_dates\block_08.pkl → 51208 rows
Saved: data_by_dates\block_09.pkl → 52480 rows
Saved: data_by_dates\block_10.pkl → 54261 rows
Saved: data_by_dates\block_11.pkl → 54502 rows
Saved: data_by_dates\block_12.pkl → 55736 rows
Saved: data_by_dates\block_13.pkl → 54795 rows
Saved: data_by_dates\block_14.pkl → 54993 rows
Saved: data_by_dates\block_15.pkl → 54159 rows
Saved: data_by_dates\block_16.pkl → 54091 rows
Saved: data_by_dates\block_17.pkl → 54053 rows
Saved: data_by_dates\block_18.pkl → 52016 rows
Saved: data_by_dates\block_19.pkl → 52263 rows
Saved: data_by_dates\block_20.pkl → 49783 rows
Saved: data_by_dates\block_21.pkl → 

In [11]:
# Plots the real and planned trajectory in a html file
def plot_both_routes_on_map(plan_route, real_route, title="Planned route VS real route"):
    center_lat, center_lon, _, _ = plan_route[0]

    fmap = folium.Map(location=[center_lat, center_lon], zoom_start=6)

    plan_points = [(lat, lon) for lat, lon, _, _  in plan_route]
    real_points = [(lat, lon) for lat, lon, _, _  in real_route]

    folium.PolyLine(plan_points, color="blue", weight=3, opacity=0.7, tooltip="Planned route").add_to(fmap)
    folium.PolyLine(real_points, color="red", weight=3, opacity=0.7, tooltip="Real route").add_to(fmap)

    folium.Marker(plan_points[0], popup="Begining planned", icon=folium.Icon(color="blue")).add_to(fmap)
    folium.Marker(real_points[0], popup="Begining real", icon=folium.Icon(color="red")).add_to(fmap)

    folium.Marker(plan_points[-1], popup="End planned", icon=folium.Icon(color="blue", icon="flag")).add_to(fmap)
    folium.Marker(real_points[-1], popup="End real", icon=folium.Icon(color="red", icon="flag")).add_to(fmap)

    return fmap

# First flight
print(df.iloc[1])

# Show both routes from the first flight in the DataFrame
mapa = plot_both_routes_on_map(
    df.iloc[0]["plan_route"],
    df.iloc[0]["real_route"],
    title="Comparación de rutas"
)

mapa.save("rutas_comparadas.html")




flight_id                                      AFL2529AA73604073
date                                         2018-03-25 00:00:00
dep_airport                                                 LEMG
arr_airport                                                 None
plan_route     [(36.675, -4.4991666666666665, 0.0, 2018-03-24...
real_route     [(36.675, -4.4991666666666665, 0.0, 2018-03-24...
Name: 1, dtype: object


In [12]:
def print_flight(real_flight, planned_flight,parsed_plan, parsed_real):
    print(f"Departure: {planned_flight[0]}")
    print(f"Arrival: {planned_flight[1]}")
    print(f"Flight ID: {planned_flight[2]}")
    print()
    print(f"Plan Route:")
    for p in parsed_plan:
        print(p)
    print(f"Real Route:")
    for p in parsed_real:
        print(p)
    print("\n")

def calc_duration(route):
    return (route[-1][3] - route[0][3]).total_seconds() / 60
   

def find_flight():
    first = 0
    for i in range(len(real_data_flights)):
        print(f"Processing day {i}...")
        
        # if i in [5, 6, 7]:
        for real_flight, planned_flight in zip(real_data_flights[i], planned_data_flights[i]):
            try:
                if (real_flight[0] == 'LEMD' and real_flight[2] == 'IBE6801AA73785151') or (real_flight[1] == 'LEMD' and real_flight[2] =='IBE6888') or (real_flight[1] == 'LEGE' and real_flight[2] == 'HBVWMAA73725877'):
                    real_parsed = [parse_coord_line(line) for line in real_flight[3].split()]
                    plan_parsed = [parse_coord_line(line) for line in planned_flight[3].split()]
                    
                    delay = calc_duration(real_parsed) - calc_duration(plan_parsed)
                    first += 1
                    print("\n")
                    print(f"Delay: {delay} minutes")
                    print_flight(real_flight, planned_flight, plan_parsed, real_parsed)
                    print("\n") 
                    
                    real_parsed = [p[:4] for p in real_parsed]
                    plan_parsed = [p[:4] for p in plan_parsed]

                    # Show both routes from the first flight in the DataFrame
                    mapa = plot_both_routes_on_map(
                        plan_parsed,
                        real_parsed,
                        title="Comparación de rutas"
                    )

                    mapa.save(f"rutas_comparadas{first}.html")
                        
                
            except Exception as e:
                print(f"Error in flight: {e}")
                continue


find_flight()

Processing day 0...
Processing day 1...
Processing day 2...
Processing day 3...
Processing day 4...


Delay: 229.26666666666665 minutes
Departure: LSZA
Arrival: LEGE
Flight ID: HBVWMAA73725877

Plan Route:
(46.00361111111111, 8.910277777777779, 0.0, datetime.datetime(2018, 3, 29, 9, 5), 'LSZA')
(45.995, 8.90611111111111, 0.4572, datetime.datetime(2018, 3, 29, 9, 5, 16), '!AKra')
(45.97777777777778, 8.898055555555555, 0.7619999999999999, datetime.datetime(2018, 3, 29, 9, 5, 40), '!AKae')
(45.969166666666666, 8.893888888888888, 1.0668, datetime.datetime(2018, 3, 29, 9, 6, 2), '!AKrb')
(45.94333333333333, 8.881666666666668, 1.5239999999999998, datetime.datetime(2018, 3, 29, 9, 6, 31), '!AKrc')
(45.87416666666667, 8.84888888888889, 2.49936, datetime.datetime(2018, 3, 29, 9, 7, 34), 'PINIK')
(45.81361111111111, 8.819166666666666, 3.3528, datetime.datetime(2018, 3, 29, 9, 8, 29), '!AKrd')
(45.761944444444445, 8.793888888888889, 3.9623999999999997, datetime.datetime(2018, 3, 29, 9, 9, 13), '!

In [13]:
infile.close()
infile2.close()


print("END")

END
